In [1]:
from utils_general import *
from utils_methods import *
import json

# from utils_methods_FedDC import train_FedDC

In [2]:
class config(object):
    def __init__(self):
        self.dataset="cifar10"
        self.device = "cuda"
        self.comm_round=2
        self.lr = 0.01
        self.batch_size = 64
        self.epochs = 1
        self.weight_decay = 1e-3
        self.n_parties = 5
        self.seed = 43
        self.alg="fedavg"
        self.rootdir="./result/bench2/"
        self.datadir="./Folder/"
        self.beta=0.1
        self.model = 'cifar10_LeNet' # Model type


args = config()

In [3]:
from utils_libs_wy import get_logger

# def get_logger(logger_path):
#     logging.basicConfig(
#         filename=logger_path,
#         # filename='/home/qinbin/test.log',
#         format='[%(asctime)s] %(levelname)s: %(message)s',
#         datefmt='%m-%d %H:%M', 
#         level=logging.DEBUG, 
#         filemode='w'
#     )

#     logger = logging.getLogger()
#     logger.setLevel(logging.INFO)
#     ch = logging.StreamHandler()
#     logger.addHandler(ch)

#     return logger 

In [4]:
seed = args.seed
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)

In [5]:
# # Dataset initialization
# data_path = args.datadir
# n_client = args.n_parties
# # Dirichlet (0.3)

# data_obj = DatasetObject(dataset=args.dataset.upper(), n_client=n_client, seed=args.seed, unbalanced_sgm=0, rule='Drichlet', rule_arg=args.beta, data_path=data_path)


In [6]:
# # Model function
# model_func = lambda : client_model(args.model)
# init_model = model_func()



tag = f"{args.dataset}-{args.model}-{args.alg}-N{args.n_parties}-beta{args.beta}-ep{args.epochs}-lr{args.lr}-round{args.comm_round}"
args.exp_dir = os.path.join(args.rootdir, tag) 
args.logdir = os.path.join(args.exp_dir, f"seed{args.seed}")
os.makedirs(args.logdir)

argument_path='experiment_arguments-%s.json' % datetime.datetime.now().strftime("%Y-%m-%d-%H:%M-%S")
with open(os.path.join(args.logdir, argument_path), 'w') as f:
    json.dump(str(args), f)

log_path= 'experiment_log-%s' % (datetime.datetime.now().strftime("%Y-%m-%d-%H:%M-%S")) + '.log'
logger = get_logger(logger_path=os.path.join(args.logdir, log_path))

# load and partition dataset
ds, data_info = get_dataset(dataset=args.dataset)  
split_distribution, dataidx_map_train = partition_labeldir(
    targets=ds['train_labels'], 
    num_classes=data_info['num_classes'], 
    n_parties=args.n_parties,
    beta=args.beta
)
client_data_train = make_client_dataset_from_partition(ds['train_data'], args.n_parties, dataidx_map_train)  
logger.info('Client dataset partitioning completed')
traindata_cls_counts = record_net_data_stats(ds['train_labels'], dataidx_map_train, logger)

model_func = lambda : ConvNet(
    channel=data_info['channel'], 
    num_classes=data_info['num_classes'], 
    net_width=128, 
    net_depth=3, 
    net_act="relu", 
    net_norm="batchnorm", 
    net_pooling="avgpooling", 
    im_size=data_info['img_size']
)
init_model = model_func()
# init_model.to(args.device)
# local_trainloaders = (DataLoader(client_data_train[i], batch_size=args.batch_size, shuffle=True) for i in range(args.n_parties))
# central_testloader = DataLoader(ds["test_data"], batch_size=128, shuffle=False)

print(f'Trying {args.alg}')
train_FedAvg(
    args,
    logger,
    init_model,
    client_data_train,
    ds["test_data"]
)      


Files already downloaded and verified
Files already downloaded and verified


Client dataset partitioning completed
Client  0 total train data: 13042, distribution: {1: 1, 2: 4990, 4: 3643, 5: 220, 6: 4188}
Client  1 total train data:  8714, distribution: {2: 1, 3: 1, 4: 1094, 5: 2960, 6: 607, 7: 4051}
Client  2 total train data: 10409, distribution: {0: 4, 1: 4996, 2: 8, 3: 4045, 4: 45, 5: 1311}
Client  3 total train data: 12305, distribution: {0: 4995, 1: 2, 3: 953, 4: 1, 5: 508, 6: 1, 7: 846, 8: 4999}
Client  4 total train data:  5530, distribution: {0: 1, 1: 1, 2: 1, 3: 1, 4: 217, 5: 1, 6: 204, 7: 103, 8: 1, 9: 5000}


Trying fedavg


Round  0: selected clients: [0, 1, 2, 3, 4]
Round  0: training client 0
Round  0: training client 1


Epoch  0, Testing Accuracy: 0.1911, Loss: 5.4193


...

Round  1: training client 3


Epoch  0, Testing Accuracy: 0.2122, Loss: 4.7696


Round  1: training client 4


Epoch  0, Testing Accuracy: 0.2972, Loss: 3.8804
Epoch  0, Testing Accuracy: 0.1659, Loss: 4.2544


>> Round  1: global model checkpoint saved
>> Round  1: Global model test loss: 2.1946
>> Round  1: Global model test accuracy: 18.81%, historical best acc: 18.81%

Session completed at 2023-07-27 01:09:44, time elapsed: 1 mins. That's all folks.
